# Figure 2

NOTE: If you're running this on **Google Colab**, then uncomment and run the next two cells.

In [ ]:
# !git clone https://github.com/Mark-Kramer/Aperiodic-Exponent-Model.git

In [ ]:
# import sys
# sys.path.insert(0,'/content/Aperiodic-Exponent-Model')

---

## Load packages

In [ ]:
import numpy as np
import scipy.io as io
import matplotlib.pyplot as plt
from   matplotlib.backends.backend_pdf import PdfPages

## Load functions for computations and plotting

In [ ]:
from one_over_f_functions import *

## Define the model

In [ ]:
# Simulate the generative model.
def wilson_cowan_with_noise(noise):
    
    c1=15;   c2=15;     c3=15;  c4=7;         # Parameters from Fig 10 of [Wilson & Cowan, 1972]
    ae=1;    thetaE=2;  ai=2;  thetaI=2.5;
    tauE=50; tauI=50;
    RE=1;    RI=1;
    ke=1;    ki=1;
    P=1.25;  Q=0;
    
    N = 60000;
    dt = 0.1;
    
    E = np.zeros([N])                         # Output variables
    I = np.zeros([N])
    t = np.arange(N)*dt/1000                  # Time in [ms]
    
    E[0] = 0.22                               # Initial conditions
    I[0] = 0.22
    
    for n in np.arange(N-1):                  # Simulate the model
        x      = c1*E[n] -c2*I[n] + P
        Se     = 1/(1+np.exp(-ae*(x-thetaE))) - 1/(1+np.exp(ae*thetaE))
        E[n+1] = E[n] + dt*1/tauE*( -E[n] + (ke-RE*E[n])*Se + noise["E"]*np.random.randn() )
        x      = c3*E[n] -c4*I[n] + Q;
        Si     = 1/(1+np.exp(-ai*(x-thetaI))) - 1/(1+np.exp(ai*thetaI));
        I[n+1] = I[n] + dt*1/tauI*( -I[n] + (ki-RI*I[n])*Si + noise["I"]*np.random.randn() )
        
    E = E[10000:]; I = I[10000:]              # Drop initial transient.
    t = t[10000:]-t[10000]                    # start time at 0 s.
    
    return E,I,t

## Figure 2A-D: Simulate the model, and plot it.

In [ ]:
noise = {"E":[],                           # Set the E noise,
         "I":0.1  }                        # ... and the I noise.
fig, ((ax1, ax2,), (ax3, ax4)) = plt.subplots(2, 2,figsize=(12, 6), dpi=80)

noise["E"]=0.0;                            # E noise = 0.
[E,I,t] = wilson_cowan_with_noise(noise)   # Simulate model,
[S,f]   = compute_spectrum(E,t)            # ... compute the spectrum & fit aperiodic exponent.
[aperiodic_exponent, x_linear_fit, y_linear_fit] = estimate_aperiodic_exponent(S,f,[20,250])
make_plots(t,E, f,S, x_linear_fit,y_linear_fit,aperiodic_exponent, ax1,ax3);
ax1.set_ylim([0.2, 0.25]); ax3.set_ylim([-20, 0]); ax3.set_xlim([0,3])

noise["E"]=0.2;                            # E noise > 0.
[E,I,t]= wilson_cowan_with_noise(noise)    # Simulate model,
[S,f]  = compute_spectrum(E,t)             # ... compute the spectrum & fit aperiodic exponent.
[aperiodic_exponent, x_linear_fit, y_linear_fit] = estimate_aperiodic_exponent(S,f,[20,250])
make_plots(t,E, f,S, x_linear_fit,y_linear_fit,aperiodic_exponent, ax2,ax4);
ax2.set_ylim([0.2, 0.25]); ax4.set_ylim([-20, 0]); ax4.set_xlim([0,3])

#fig.savefig("./PDFs/Figure-2ABCD.pdf", bbox_inches='tight')

## Figure 2E (Part 1): Iterate over values of noise and plot aperiodic exponent
This part is slow.

In [ ]:
noise = {"E":[],                                           # Prepare the noise variable.
         "I":0.1  }                                        # ... fix the I-noise,
noise_values = np.arange(0.0,0.31, 0.01)                    # ... define range of E-noises.
N_replicates = 100                                         # Number of times to repeat simulation.
aperiodic_exponents = np.zeros([np.size(noise_values), N_replicates])

for [count, E_noise] in enumerate(noise_values):           # For each E-noise,
    noise["E"]=E_noise
    print("E_noise: %.4f" % E_noise)
    for j in np.arange(N_replicates):
        [E,I,t]= wilson_cowan_with_noise(noise)            # Simulate model N_replicate times,
        [S,f]  = compute_spectrum(E,t)                     # ... compute the spectrum,
        ae = estimate_aperiodic_exponent(S,f,[20,250])[0]  # ... fit aperiodic exponent,
        aperiodic_exponents[count, j] = ae                 # ... and save it.
                                                           # Save the results.
res = {"noise_values":noise_values, "aperiodic_exponents":aperiodic_exponents}
io.savemat(str('Figure-2E.mat'), res)

## Figure 2E (Part 2): Load the aperiodic exponent, and plot it.

This part is fast.

In [ ]:
# Note: if you're running on Google Colab, then load the .mat file like this:
# res = io.loadmat("/content/Aperiodic-Exponent-Model/Figure-2E.mat");

# Or, if not running on Google Colab, then load the .mat file like this:
res = io.loadmat("Figure-2E.mat");  res["color"] = "red"

fig = plt.figure(figsize=(12, 4), dpi=80)
make_plot_ae_vs_noise(res)

#fig.savefig("./PDFs/Figure-2E.pdf", bbox_inches='tight')